# 0.0. IMPORTS

In [40]:
import pandas as pd
import inflection
import math

## 0.1.0 Helper Functions

## 0.2.0. Loading Data

In [28]:
# arquivo com as vendas das lojas
df_sales_raw = pd.read_csv( 'data/train.csv', low_memory = False )

# arquivo com as caracteristicas das lojas
df_store_raw = pd.read_csv( 'data/store.csv', low_memory = False )

# merge com base na chave comum "Store"
df_raw = pd.merge( df_sales_raw, df_store_raw, how = 'left', on = 'Store' )

# 1.0. DESCRIÇÃO DOS DADOS

In [83]:
df1 = df_raw.copy()

## Explicação das colunas

* store - ID único para cada loja


* day_of_week - dia da semana em números inteiros


* date - data que foi feita a venda


* sales - valor das vendas em determinado dia


* customers - número de clientes em determinado dia


* open - indicador se a loja estava aberta e fechada 0 = closed, 1 = open


* promo - Indica se a loja estava com promoção naquele dia


* state_holiday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None


* school_holiday - indicates if the (Store, Date) was affected by the closure of public schools



* store_type - differentiates between 4 different store models: a, b, c, d


* assortment - describes an assortment level: a = basic, b = extra, c = extended


* competition_distance - distance in meters to the nearest competitor store


* competition_open_since_month - gives the approximate month of the time the nearest competitor was opened


* competition_open_since_year - gives the approximate year of the time the nearest competitor was opened


* promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating



* promo2_since_week - describes the week when the store started participating in Promo2


* promo2_since_year - describes the year when the store started participating in Promo2


* promo_interval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store



## 1.1. Rename Columns

In [84]:
# crio uma lista com o nome das colunas antigos
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
           'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceMonth',
           'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
           'Promo2SinceYear', 'PromoInterval']

# coloco as colunas no estilo snake_case
snake_case = lambda x : inflection.underscore( x ) # defino a lambda function

# aplico o map para cada nome cols_old e crio a lista cols_new
cols_new = list( map( snake_case, cols_old ) )

# renomeio as coluanas do dataset
df1.columns = cols_new

## 1.2. Data Dimensions

In [85]:
print( 'Number of rows {}'.format( df1.shape[0] ) )
print( 'Number of columns {}'.format( df1.shape[1] ) )

Number of rows 1017209
Number of columns 18


## 1.3. Data Types

In [87]:
# alterando o tipo da coluna date de int para date 64 
df1['date'] = pd.to_datetime( df1['date'] )

# exibindo os tipos de dados
print( df1.dtypes )

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object


## 1.4. Check NA

In [88]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5. Fillout NA

In [92]:
#competition_distance - Considero que na é igual a não competição. Se eu subsituir na por uma distancia muito grande seria o mesmo que dizer que não existe competiçã ( o valor máximo df1['competition_distance'] é 75860.0 )
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000 if math.isnan( x ) else x )


#competition_open_since_month - Considero que quando tiver na o competidor abriu no mesmo mes da coluna date ( data da venda ) -- para cada linha na pego o mes da coluna date e substituo na coluna competition_open_since_month
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else x['competition_open_since_month'], axis = 1 )

#competition_open_since_year - mesma lógica do competition_open_since_month mas aplicado ao ano
df1['competition_open_since_year'] = df1.apply( lambda x : x['date'].year if math.isnan( x['competition_open_since_year']                                                 ) else x['competition_open_since_year'], axis = 1)


#promo2_since_week - Vou considerar que na a loja não participou da promoção. Então vou substituir o na pela semana da data na coluna date ( data da venda)
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan( x['promo2_since_week'] ) else x['promo2_since_week'] , axis = 1)

#promo2_since_year - mesma lógica da promo2_since_week só que substituindo pelo ano
df1['promo2_since_year'] = df1.apply( lambda x : x['date'].year if math.isnan( x['promo2_since_year'] ) else x['promo2_since_year'] , axis = 1 )

#promo_interval
month_map = { 1 : 'Jan', 2 :  'Feb', 3 : 'Mar', 4 : 'Apr', 5 : 'May', 6 : 'Jun', 7 : 'Jul', 8 : 'Aug', 9 : 'Sept', 10 : 'Oct', 11 : 'Nov', 12 : 'Dec'  } # crio um dicionario com os meses de acordo com os valores da coluna promo_interval

df1['month_map'] = df1['date'].dt.month.map( month_map ) # substituo o numero do mes pelo nome

df1['promo_interval'].fillna( 0 , inplace = True ) # substituo os na por 0

df1['is_promo'] = df1[['promo_interval','month_map']].apply(lambda x : 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis = 1) # para cada linha verifico se o valor da coluna month_map existe na coluna promo_interval, se existir retorno 1 ( participa da promoção ) se não existir 0 ( não participa )



# .0

# .0

# .0

# .0